# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [ ]:
# Your code goes here
import pandas as pd 
df1 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv')
df3 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv')
# pandas has no pd.head; call head() on each DataFrame (or print them)
print("df1 head:")
print(df1.head())
print("\ndf2 head:")
print(df2.head())
print("\ndf3 head:")
print(df3.head())


### Concatenate into single DataFrame 

In [ ]:
Combined_df = pd.concat([df1, df2, df3], ignore_index=True)

### Applying Cleaning Functions 

In [ ]:
def clean_data(df):
    df.col = df.col.str.lower().str.strip()
    return df

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

#### Load Data

In [ ]:
# Your code goes here
import pandas as pd 
df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv')

### Initial Exploration 

In [13]:
print(df.head())
df.info()
print(df.isnull().sum())

   unnamed:_0 customer       state  customer_lifetime_value response  \
0           0  DK49336     Arizona              4809.216960       No   
1           1  KX64629  California              2228.525238       No   
2           2  LZ68649  Washington             14947.917300       No   
3           3  XL78013      Oregon             22332.439460      Yes   
4           4  QA50777      Oregon              9025.067525       No   

   coverage education effective_to_date employmentstatus gender  ...  \
0     Basic   College        2011-02-18         Employed      M  ...   
1     Basic   College        2011-01-18       Unemployed      F  ...   
2     Basic  Bachelor        2011-02-10         Employed      M  ...   
3  Extended   College        2011-01-11         Employed      M  ...   
4   Premium  Bachelor        2011-01-17    Medical Leave      F  ...   

   number_of_policies     policy_type        policy  renew_offer_type  \
0                   9  Corporate Auto  Corporate L3          

### Standardize and Format Columns 

In [ ]:

df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], errors='coerce')


### Handle Missing Values

In [ ]:
df = df.fillna({'response':'No Response'})    # Example for a categorical
df = df.dropna(subset=['customer'])           # Drop rows with missing key values


#### Feature Engineering & Structuring


In [17]:
# Create age bands if a customer_age column exists; otherwise create a safe fallback
if 'customer_age' in df.columns:
	df['age_band'] = pd.cut(df['customer_age'], bins=[0,29,49,120], labels=['<30','30-49','50+'])
else:
	# customer_age not available — try a proxy using months_since_policy_inception (years)
	if 'months_since_policy_inception' in df.columns:
		df['age_years_proxy'] = (df['months_since_policy_inception'] // 12).astype(int)
		df['age_band'] = pd.cut(df['age_years_proxy'], bins=[-1,29,49,120], labels=['<30','30-49','50+'])
		df.drop(columns=['age_years_proxy'], inplace=True)
	else:
		# final fallback: set a single 'Unknown' band so grouping runs without KeyError
		df['age_band'] = 'Unknown'

# compute tenure in years (uses effective_to_date which was already converted to datetime)
df['tenure_years'] = (pd.Timestamp('now') - df['effective_to_date']).dt.days // 365

# group by the age_band and compute mean total_claim_amount and customer_lifetime_value
df_grouped = df.groupby('age_band').agg({'total_claim_amount':'mean', 'customer_lifetime_value':'mean'}).reset_index()

/var/folders/00/c54qzf757tzdyqs5hr4n94vc0000gn/T/ipykernel_19889/3473632412.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_grouped = df.groupby('age_band').agg({'total_claim_amount':'mean', 'customer_lifetime_value':'mean'}).reset_index()


### Clean and Unify Categorical Values

In [18]:
df['gender'] = df['gender'].str.title().replace({'M': 'Male', 'F': 'Female'})
df['education'] = df['education'].str.title().replace({'Bachelor': 'Bachelors', 'Master': 'Masters'})

# Standardize vehicle_class: ensure title case and normalize common variants
df['vehicle_class'] = df['vehicle_class'].str.title()

# Fix known variations (after title casing, 'Suv' may appear instead of 'SUV')
df['vehicle_class'] = df['vehicle_class'].replace({
	'Two-Door': 'Two-Door Car',
	'Four-Door': 'Four-Door Car',
	'Suv': 'SUV',
	'Luxury Car': 'Luxury Car',
	'Four-Door Car': 'Four-Door Car',
	'Two-Door Car': 'Two-Door Car'
})

### Remove Duplicates

In [19]:
df = df.drop_duplicates(subset='customer')
print(f"DataFrame shape after removing duplicates: {df.shape}")

DataFrame shape after removing duplicates: (9134, 29)


### Reorder Columns

In [21]:
# select desired columns but keep only those that exist in the DataFrame to avoid KeyError
desired_cols = ['customer', 'customer_age', 'age_band', 'gender', 'income', 'policy_type', 'tenure_years', 'vehicle_type', 'response']
cols = [c for c in desired_cols if c in df.columns]

if len(cols) < len(desired_cols):
	missing = sorted(set(desired_cols) - set(cols))
	print(f"Warning: the following columns are missing and will be skipped: {missing}")

df = df[cols]
print(df.head())

  customer age_band  gender  income     policy_type  tenure_years  \
0  DK49336      <30    Male   48029  Corporate Auto            14   
1  KX64629      <30  Female       0   Personal Auto            14   
2  LZ68649      <30    Male   22139   Personal Auto            14   
3  XL78013      <30    Male   49078  Corporate Auto            14   
4  QA50777      <30  Female   23675   Personal Auto            14   

  vehicle_type response  
0            A       No  
1            A       No  
2            A       No  
3            A      Yes  
4            A       No  


### Export Structured Dataset and Proceed with Analysis

In [22]:
df.to_csv('structured_marketing_customers.csv', index=False)
#!/usr/bin/env python3

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [26]:
# Use the already loaded dataframe `df` in the notebook (do not re-import pandas here)
# Standardize sales channel names and ensure the revenue column is numeric.
# The dataset does not contain 'total_revenue' — use 'total_claim_amount' as the revenue metric.

if 'sales_channel' not in df.columns:
    raise KeyError("sales_channel column not found in df")

df['sales_channel'] = df['sales_channel'].astype(str).str.strip().str.title()

revenue_col = 'total_claim_amount'
if revenue_col not in df.columns:
    raise KeyError(f"'{revenue_col}' column not found in df")

df[revenue_col] = pd.to_numeric(df[revenue_col], errors='coerce')

# Create the pivot table summing revenue by sales channel
pivot_table = df.pivot_table(
    index='sales_channel',
    values=revenue_col,
    aggfunc='sum'
).round(2)

pivot_table = pivot_table.sort_values(by=revenue_col, ascending=False)

print(pivot_table)


               total_claim_amount
sales_channel                    
Agent                  1810226.82
Branch                 1301204.00
Call Center             926600.82
Web                     706600.04


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [27]:
import pandas as pd

# Load data from the provided link
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv'
df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv')

# Standardize text in gender and education columns
df['gender'] = df['gender'].str.title().str.strip()
df['education'] = df['education'].str.title().str.strip()
df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')

# Create the pivot table
pivot_table = df.pivot_table(
    index='gender',
    columns='education',
    values='customer_lifetime_value',
    aggfunc='mean'
).round(2)

print(pivot_table)


education  Bachelor  College   Doctor  High School Or Below   Master
gender                                                              
F           7874.27  7748.82  7328.51               8675.22  8157.05
M           7703.60  8052.46  7415.33               8149.69  8168.83


## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

In [30]:
import pandas as pd

# Load the dataset
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv'
df = pd.read_csv(url)

# Determine month for complaints:
# - If there's a 'complaint_date' column parse it
# - Otherwise fall back to the existing 'month' integer column (1-12)
if 'complaint_date' in df.columns:
    df['complaint_date'] = pd.to_datetime(df['complaint_date'], errors='coerce')
    df['complaint_month'] = df['complaint_date'].dt.month_name()
elif 'month' in df.columns:
    month_map = {
        1: 'January', 2: 'February', 3: 'March', 4: 'April',
        5: 'May', 6: 'June', 7: 'July', 8: 'August',
        9: 'September', 10: 'October', 11: 'November', 12: 'December'
    }
    df['complaint_month'] = df['month'].map(month_map)
else:
    raise KeyError("Neither 'complaint_date' nor 'month' column found in df. Cannot derive complaint month.")

# Standardize text for policy type
if 'policy_type' in df.columns:
    df['policy_type'] = df['policy_type'].astype(str).str.title().str.strip()
else:
    raise KeyError("'policy_type' column not found in df")

# Find a complaints column (datasets sometimes use different names)
complaint_candidates = [
    'number_of_open_complaints',
    'Number of Open Complaints',
    'number_of_complaints',
    'complaints'
]
complaint_col = next((c for c in complaint_candidates if c in df.columns), None)
if complaint_col is None:
    raise KeyError("No complaints column found in df. Looked for: " + ", ".join(complaint_candidates))

# Ensure complaints are numeric
df[complaint_col] = pd.to_numeric(df[complaint_col], errors='coerce').fillna(0).astype(int)

# Aggregate to get total number of complaints by policy_type and month (long format)
complaint_summary = (
    df
    .groupby(['policy_type', 'complaint_month'], as_index=False)[complaint_col]
    .sum()
    .rename(columns={complaint_col: 'num_complaints'})
    .sort_values(['policy_type', 'num_complaints'], ascending=[True, False])
)

# Drop rows where month is missing (if any)
complaint_summary = complaint_summary.dropna(subset=['complaint_month'])

print(complaint_summary)


      policy_type complaint_month  num_complaints
1  Corporate Auto         January             415
0  Corporate Auto        February             361
3   Personal Auto         January            1635
2   Personal Auto        February            1363
4    Special Auto        February              91
5    Special Auto         January              84


*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [ ]:
# Your code goes here